In [ ]:
# Step 1: load fb_friends and bt_symmetric csv files
# Step 2: Preprocess, drop the rows in bt_symmetric containing:
#    * Empty scans are marked with user B = -1 and RSSI = 0
#    * Scans of devices outside of the experiment are marked with user B = -2. All non-experiment devices are given the same ID.
# Step 3: Use the algorithm to get a confidence score for the association fb_friends -> friends irl
    friends_irl_score(a, b) = total_ping_count(a, b)/rssi_total(a, b)
    Its up to us to define what we consider as "friends". Are persons a and b friends if they have met once over the study (30 days)? Are they friends if they have met twice?
    Its up to us to define it and then assume the definition for our own data mining study.
# Step 4: Plot graphs
# Step 5: Interpret the results